In [38]:
import nltk
nltk.download('all')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

import matplotlib.pyplot as plt
import re
import numpy as np


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [21]:
import pandas as pd
data = pd.read_csv('/content/sample_data/post_yellowknife_evacuation - Sheet1.csv')
print(data.head())

                                           poster  \
0                       Yellowknife House Sitting   
1       Yellowknife's Rants & Raves 2.0 (Rated E)   
2  Families With Pets Evacuating Yellowknife 2023   
3                                      Gracie Ann   
4                                  Alannah Monica   

                                             Content  Likes & loves  \
0  H here!i became a full-time housesitter as of ...           66.0   
1  Are evacuation insurance claims will result in...            2.0   
2  Hey folks, a number of crates were loaned out ...            4.0   
3  Volunteers preparing food hampers Edit! These ...            NaN   
4  Thank you guys for the concern on the Yellowkn...           66.0   

      comments       shares  
0  21 comments     2 shares  
1  28 comments     2 shares  
2   2 comments     5 shares  
3  57 comments    24 shares  
4  15 comments  15 comments  


In [ ]:
print(data['Likes & loves'].isna().sum())
print(data['poster'].isna().sum())

20
4


In [22]:
# fillna values for likes & loves column and poster column using iloc function
data.iloc[:, 3:4] = data.iloc[:, 3:4].fillna(0)
data.iloc[:, 0:1] = data.iloc[:, 0:1].fillna('Anonymous')
print(data.iloc[:, 3:4].isna().sum())
print(data.iloc[:, 5:6].isna().sum())

comments    0
dtype: int64
Series([], dtype: float64)


In [ ]:
# prompt: remove special characters using regular expressions in python

import re

# Remove special characters from the 'Text' column
data['Text'] = data['Text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

# Print the updated DataFrame
print(data.head())


In [24]:
def preprocess_text(text):

    # Tokenize the text

    tokens = word_tokenize(text.lower())




    # Remove stop words

    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]




    # Lemmatize the tokens

    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]




    # Join the tokens back into a string

    processed_text = ' '.join(lemmatized_tokens)

    return processed_text

In [30]:
data['Content'] = data['Content'].apply(preprocess_text)
data['Content'] = data['Content'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
corpus = [i for i in data['Content']]
print(corpus)

['h  became fulltime housesitter august st  evacuation taking lot time   looking continue long possible current availability   see', 'evacuation insurance claim result increased policy premium renewal  deductables  would good list answer various insurer ', 'hey folk  number crate loaned yellowknife evacuation return information written  anyone need returned edmonton  people hoping get crate back  please post help organize return trip  everyone', 'volunteer preparing food hamper edit  hamper rae resident  hamper planned place help rae resident lost grocery evacuation due power outagesthese hamper already planned started yellowknife evacuation happened ', 'thank guy concern yellowknife evacuationi m happy re thinking  clarify though  edmonton closer yellowknife   hour drive  u   hour drive   see', ' put together timeline experience evacuation wanted share publicly know others shared concern unhoused community member   critical moment timeline  find full detail website  http  wwwykwscapos

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.corpus import stopwords

In [32]:
count_vect = CountVectorizer(stop_words=stopwords.words('english'), lowercase=True)
x_counts = count_vect.fit_transform(corpus)
x_counts.todense()
print(count_vect.get_feature_names_out())

['ab' 'able' 'access' ... 'ykws' 'yzf' 'zf']


In [33]:
tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_counts)

In [34]:
dimension = 5
lda = LDA(n_components = dimension)
lda_array = lda.fit_transform(x_tfidf)
print(lda_array)

[[0.83488965 0.04128797 0.04128135 0.04109635 0.04144468]
 [0.04185558 0.04180539 0.83250671 0.04187365 0.04195866]
 [0.85488584 0.03639944 0.03642293 0.03611921 0.03617257]
 [0.04344119 0.04287815 0.04294847 0.04319628 0.82753591]
 [0.04515523 0.81996969 0.04511644 0.0449485  0.04481015]
 [0.035966   0.03584924 0.85627418 0.03587169 0.0360389 ]
 [0.05834053 0.05831156 0.05863665 0.05827322 0.76643805]
 [0.03097963 0.03088569 0.8764932  0.03075212 0.03088936]
 [0.02824831 0.02804797 0.0286948  0.887287   0.02772191]
 [0.05147387 0.05140918 0.79487576 0.05126516 0.05097602]
 [0.03148186 0.03126612 0.03111944 0.03122936 0.87490322]
 [0.03310647 0.03221676 0.03277288 0.86963858 0.0322653 ]
 [0.0295461  0.02874724 0.88410688 0.02872197 0.02887779]
 [0.04776399 0.04773641 0.80916489 0.04764129 0.04769341]
 [0.04668182 0.81301816 0.04689564 0.04680021 0.04660418]
 [0.03840877 0.03759572 0.03820279 0.84801996 0.03777276]
 [0.03067509 0.88041944 0.02974135 0.02950276 0.02966136]
 [0.87554488 0

In [39]:
components = [lda.components_[i] for i in range(len(lda.components_))]
features = count_vect.get_feature_names_out()
important_words = [sorted(features, key = lambda x: components[j][np.where(features == x)[0][0]], reverse = True)[:3] for j in range(len(components))]
print(important_words)

[['resident', 'return', 'yellowknife'], ['yellowknife', 'evacuation', 'able'], ['evacuation', 'yellowknife', 'city'], ['today', 'homeless', 'yellowknife'], ['yellowknife', 'evacuation', 'available']]
